In [13]:
from data_loader.scrape_zakonik import scrape_zakonik
from data_loader.load import load_px
from IPython.display import display, HTML, Markdown
import json

zakonik = scrape_zakonik()
print(len(zakonik))

211


In [12]:
# md: str = ""

# for KD, alineje in zakonik.items():
#     if KD == "kršitev spolne nedotakljivost z zlorabo položaja":
#         KD = "kršitev spolne nedotakljivosti z zlorabo položaja"
#     md += f"## {KD}:\n"
#     if not alineje:
#         md += f"* No data available *\n"
#         continue
#     for alineja in alineje:
#         md += f"- {alineja}\n"
# display(Markdown(md))


In [3]:
kd_min_max_jail: dict = {}

In [4]:
sankcije = load_px()

#### Pretvorba stolpca "KAZNOVO DEJANJE" v nam uporaben format

In [5]:
sankcije_kd_formated = sankcije["KAZNIVO DEJANJE"]
def format_kd(x):
  try:
    int(x[0])
  except ValueError:
    return x.strip().lower()
  return x[x.find(" "): ].strip().lower()
sankcije_kd_formated = sankcije_kd_formated.apply(format_kd)
sankcije["KAZNIVO DEJANJE"] = sankcije_kd_formated

In [6]:
# sankcije.where(sankcije["KAZNIVO DEJANJE"] == "umor".lower()).dropna().where(sankcije["GLAVNA KAZEN"] != "Glavna kazen - SKUPAJ").dropna().where(sankcije["SPOL"] == "Spol - SKUPAJ").dropna()

In [7]:
# map sankcije["GLAVNA KAZEN"] to jailtime in years
# Middle of range jailtimes or just the number that is stated if not a range
jail_time_for_kazen: dict = {
    '11100 Zaporna kazen - nad 2 do 10 let (nepogojna)': 6,
    '11101 Zaporna kazen - nad 10 let  (nepogojna)': 10,
    '11105 Zaporna kazen - nad 1 do 2 leti (nepogojna)': 1.5,
    '11106 Zaporna kazen - nad 6 mes. do 1 leta (nepogojna)': 0.75,
    '11107 Zaporna kazen - nad 3 do 6 mes. (nepogojna)': 0.375,
    '11108 Zaporna kazen - nad 2 do 3 mes. (nepogojna)': 0.208,
    '11109 Zaporna kazen - nad 1 do 2 mes. (nepogojna)': 0.125,
    '11110 Zaporna kazen - do 30 dni (nepogojna)': 0.042,

    '11205 Zaporna kazen - nad 1 do 2 leti (pogojna)': 1.5,
    '11206 Zaporna kazen - nad 6 mes. do 1 leta (pogojna)': 0.75,
    '11207 Zaporna kazen - nad 3 do 6 mes. (pogojna)': 0.375,
    '11208 Zaporna kazen - nad 2 do 3 mes. (pogojna)': 0.208,
    '11209 Zaporna kazen - nad 1 do 2 mes. (pogojna)': 0.125,
    '11210 Zaporna kazen - do 30 dni (pogojna)': 0.042,

    # No Jailtime penalties
    '120 Denarna kazen (nepogojna)': 0,
    '1205 Denarna kazen (pogojna)': 0,
    '13 Sodni opomin': 0,
    '14 Kazen opuščena': 0
}


In [8]:
#! NOT VERY EFFICIENT (LMAO)
# Calculate average jail time for each KD
jail_time_for_kd: dict = {}
for KD, alineje in zakonik.items():
    if KD == "kršitev spolne nedotakljivost z zlorabo položaja":
        KD = "kršitev spolne nedotakljivosti z zlorabo položaja"
    data = sankcije.where(
        sankcije["KAZNIVO DEJANJE"] == KD).dropna().where(
            sankcije["SPOL"] == "Spol - SKUPAJ").dropna().where(
                sankcije["GLAVNA KAZEN"].isin(jail_time_for_kazen.keys())).dropna()
    jailtime = data["GLAVNA KAZEN"].apply(lambda x: jail_time_for_kazen[x]).mean()
    jail_time_for_kd[KD] = jailtime

In [9]:
kazni_dict: dict = {}


class DataFormat:
    max_jailtime: float
    average_actual_jailtime: float
    dangerousness: float = 1
    """dangerousness is a weight based on how dangerous the KD is for a person# """

    def __init__(self, max_jailtime: float, average_actual_jailtime: float, dangerousness=1):
        self.max_jailtime = max_jailtime
        self.average_actual_jailtime = average_actual_jailtime
        self.dangerousness = dangerousness

    def as_dict(self) -> dict:
        return {
            "max_jailtime": self.max_jailtime,
            "average_actual_jailtime": self.average_actual_jailtime,
            "dangerousness": self.dangerousness
        }


def kazni_dict_entry(kd: str, max_jailtime: float, average_actual_jailtime_dict: dict, dangerousness=1) -> None:
    data = DataFormat(max_jailtime, average_actual_jailtime_dict.get(kd), dangerousness)
    kazni_dict[kd] = data.as_dict()


In [10]:
# MAKE ENTRIES INTO kazni_dict like this:
# kazni_dict_entry("IME KD", max_jailtime: float, jail_time_for_kd, dangerousness:float=[0, 1])

# MAX JAILTIME for life sentence is 100years!
 
kazni_dict_entry("dajanje daril za nezakonito posredovanje", 6, jail_time_for_kd, 0.1)
kazni_dict_entry("dajanje podkupnine", 6, jail_time_for_kd, 0.1)
kazni_dict_entry("davčna zatajitev", 12, jail_time_for_kd, 0)
kazni_dict_entry("detomor", 3, jail_time_for_kd, 0)
kazni_dict_entry("dogovor za kaznivo dejanje", 1, jail_time_for_kd, 0)
kazni_dict_entry("dvojna zakonska zveza", 1, jail_time_for_kd, 0)
kazni_dict_entry("financiranje terorizma", 15, jail_time_for_kd, 0)

kazni_dict_entry("genocid", 100, jail_time_for_kd, 0) #lifetime sentance

kazni_dict_entry("goljufija", 10, jail_time_for_kd, 0.25)
kazni_dict_entry("goljufija na škodo evropske unije", 8, jail_time_for_kd, 0)
kazni_dict_entry("grožnja", 1, jail_time_for_kd, 0.1)
kazni_dict_entry("huda telesna poškodba", 10, jail_time_for_kd, 1)
kazni_dict_entry("hudodelsko združevanje", 8, jail_time_for_kd, 0)
kazni_dict_entry("hudodelstva zoper človečnost", 15, jail_time_for_kd, 0)
kazni_dict_entry("hujskanje k upiranju", 3, jail_time_for_kd, 0.1)
kazni_dict_entry("izdaja in neupravičena pridobitev poslovne skrivnosti", 5, jail_time_for_kd, 0)
kazni_dict_entry("izdaja in uporaba lažnega zdravniškega ali veterinarskega spričevala", 3, jail_time_for_kd, 0)
kazni_dict_entry("izdaja tajnih podatkov", 8, jail_time_for_kd, 0)
kazni_dict_entry("izdelava, pridobitev in odtujitev pripomočkov za ponarejanje", 2, jail_time_for_kd, 0)
kazni_dict_entry("izdelovanje in pridobivanje orožja in pripomočkov, namenjenih za kaznivo dejanje", 2, jail_time_for_kd, 0)
kazni_dict_entry("izneverjenje", 5, jail_time_for_kd, 0)
kazni_dict_entry("izsiljevanje", 10, jail_time_for_kd, 0.3)
kazni_dict_entry("izsiljevanje izjave", 8, jail_time_for_kd, 0)
kazni_dict_entry("javno spodbujanje sovraštva, nasilja ali nestrpnosti", 5, jail_time_for_kd, 0.1)
kazni_dict_entry("jemanje podkupnine", 8, jail_time_for_kd, 0.1)
kazni_dict_entry("kriva izpovedba", 8, jail_time_for_kd, 0)
kazni_dict_entry("kriva ovadba", 3, jail_time_for_kd, 0.1)
kazni_dict_entry("krvoskrunstvo", 2, jail_time_for_kd, 0)
kazni_dict_entry("kršitev avtorskih sorodnih pravic", 8, jail_time_for_kd, 0)
kazni_dict_entry("kršitev družinskih obveznosti", 2, jail_time_for_kd, 0)
kazni_dict_entry("kršitev enakopravnosti", 3, jail_time_for_kd, 0.3)
kazni_dict_entry("kršitev materialnih avtorskih pravic", 8, jail_time_for_kd, 0)
kazni_dict_entry("kršitev moralnih avtorskih pravic", 1, jail_time_for_kd, 0)
kazni_dict_entry("kršitev nedotakljivosti stanovanja", 5, jail_time_for_kd, 0.3)
kazni_dict_entry("kršitev omejevalnih ukrepov", 5, jail_time_for_kd, 0)
kazni_dict_entry("kršitev pravic do sodelovanja pri upravljanju in kršitev sindikalnih pravic", 1, jail_time_for_kd, 0)
kazni_dict_entry("kršitev pravic iz socialnega zavarovanja", 1, jail_time_for_kd, 0)
kazni_dict_entry("kršitev pravic pri zaposlovanju ali brezposelnosti", 1, jail_time_for_kd, 0)
kazni_dict_entry("kršitev pravice do pravnega sredstva ali peticije", 2, jail_time_for_kd, 0)
kazni_dict_entry("kršitev proste odločitve volivcev", 2, jail_time_for_kd, 0.1)
kazni_dict_entry("kršitev spolne nedotakljivosti z zlorabo položaja", 8, jail_time_for_kd, 0.5)
kazni_dict_entry("kršitev svobodne opredelitve", 1, jail_time_for_kd, 0.1)
kazni_dict_entry("kršitev tajnosti občil", 5, jail_time_for_kd, 0.1)

# MARK LOBODA
kazni_dict_entry("kršitev tajnosti postopka", 5, jail_time_for_kd, 0.1)
kazni_dict_entry("kršitev temeljnih pravic delavcev", 5, jail_time_for_kd, 0.4)
kazni_dict_entry("kršitev volilne pravice", 1, jail_time_for_kd, 0.7)
kazni_dict_entry("kršitev človeškega dostojanstva z zlorabo uradnega položaja ali uradnih pravic", 3, jail_time_for_kd, 0.9)
kazni_dict_entry("lahka telesna poškodba", 3, jail_time_for_kd, 0.7)
kazni_dict_entry("lažno izdajanje za uradno ali vojaško osebo", 1, jail_time_for_kd, 0.3)
kazni_dict_entry("malomarno opravljanje lekarniške dejavnosti", 2, jail_time_for_kd, 1)
kazni_dict_entry("malomerno zdravljenje in opravljanje zdravilniske dejavnosti", 3, jail_time_for_kd, 1)
kazni_dict_entry("mazaštvo", 10, jail_time_for_kd, 0.1)
kazni_dict_entry("mučenje", 12, jail_time_for_kd, 1)
kazni_dict_entry("mučenje živali", 3, jail_time_for_kd, 0)
kazni_dict_entry("napad na informacijski sistem", 5, jail_time_for_kd, 0.1)
kazni_dict_entry("napad na uradno osebo, ko opravlja naloge varnosti", 5, jail_time_for_kd, 0.9)
kazni_dict_entry("napad na varnost zračnega prometa", 15, jail_time_for_kd, 1)
kazni_dict_entry("napeljavanje k samomoru in pomoč pri samomoru", 15, jail_time_for_kd, 0.1)
kazni_dict_entry("nasilje v družini", 5, jail_time_for_kd, 0)
kazni_dict_entry("nasilništvo", 3, jail_time_for_kd, 1)
kazni_dict_entry("nedovoljena proizvodnja in promet orožja ali eksploziva", 10, jail_time_for_kd, 0.1)
kazni_dict_entry("nedovoljeno dajanje daril", 6, jail_time_for_kd, 0)
kazni_dict_entry("nedovoljeno sprejemanje daril", 6, jail_time_for_kd, 0)
kazni_dict_entry("neodvrnitev nevarnosti", 2, jail_time_for_kd, 0.1)
kazni_dict_entry("neplačevanje preživnine", 3, jail_time_for_kd, 0)
kazni_dict_entry("neupravičena izdaja poklicne skrivnosti", 1, jail_time_for_kd, 0.2)
kazni_dict_entry("neupravičena osebna preiskava", 2, jail_time_for_kd, 0)
kazni_dict_entry("neupravičena proizvodnja in promet s prepovedanimi drogami, nedovoljenimi snovmi v športu in predhod", 15, jail_time_for_kd, 0)
kazni_dict_entry("neupravičena uporaba uje oznake ali modela", 3, jail_time_for_kd, 0)
kazni_dict_entry("neupravičena uporaba tujega izuma ali topografije", 3, jail_time_for_kd, 0)
kazni_dict_entry("neupravičeno prisluškovanje in zvočno snemanje", 5, jail_time_for_kd, 0.1)
kazni_dict_entry("neupravičeno slikovno snemanje", 5, jail_time_for_kd, 0.1)
kazni_dict_entry("nevarna vožnja v cestnem prometu", 12, jail_time_for_kd, 0.4)
kazni_dict_entry("nevestna veterinarska pomoč", 1, jail_time_for_kd, 0)
kazni_dict_entry("nevestno delo v službi", 3, jail_time_for_kd, 0)
kazni_dict_entry("nezakonit lov", 2, jail_time_for_kd, 0)
kazni_dict_entry("nezakonit ribolov", 1, jail_time_for_kd, 0)

# FILIP NAPREJ od nezakonito dajanje pravne pomoči
kazni_dict_entry("nezakonito dajanje pravne pomoči", 1, jail_time_for_kd, 0)
kazni_dict_entry("nezakonito ravnanje z zaščitenimi živalmi in rastlinami", 5, jail_time_for_kd, 0)
kazni_dict_entry("oborožen upor", 15, jail_time_for_kd, 0.1)
kazni_dict_entry("obrekovanje", 2, jail_time_for_kd, 0.1)
kazni_dict_entry("obremenjevanje in uničevanje okolja", 12, jail_time_for_kd, 0.1)
kazni_dict_entry("oderuštvo", 3, jail_time_for_kd, 0)
kazni_dict_entry("odklonitev sodelovanja pri odvračanju splošne nevarnosti", 0.5, jail_time_for_kd, 0)
kazni_dict_entry("odstranitev ali poškodovanje uradnega pečata ali znamenja", 0.5, jail_time_for_kd, 0)
kazni_dict_entry("odvzem ali uničenje uradnega pečata ali uradnih spisov", 2, jail_time_for_kd, 0)
kazni_dict_entry("odvzem mladoletne osebe", 5, jail_time_for_kd, 1)
kazni_dict_entry("odvzem motornega vozila", 3, jail_time_for_kd, 1)
kazni_dict_entry("ogrožanje javnega prometa z nevarnim dejanjem ali sredstvom", 12, jail_time_for_kd, 1)
kazni_dict_entry("ogrožanje okolja s hrupom ali svetlobo", 2, jail_time_for_kd, 1)
kazni_dict_entry("ogrožanje oseb pod mednarodnim varstvom", 15, jail_time_for_kd, 0.2)
kazni_dict_entry("ogrožanje posebnih vrst javnega prometa", 8, jail_time_for_kd, 1)
kazni_dict_entry("ogrožanje varnosti pri delu", 12, jail_time_for_kd, 0)
kazni_dict_entry("ogrožanje z nevarnim orodjem pri pretepu ali prepiru", 0.5, jail_time_for_kd, 1)
kazni_dict_entry("omogočanje bega osebi, ki ji je vzeta prostost",5, jail_time_for_kd, 0)
kazni_dict_entry("omogočanje uživanja prepovedanih drog ali nedovoljenih snovi v športu", 12, jail_time_for_kd, 0)
kazni_dict_entry("onesnaženje pitne vode", 12, jail_time_for_kd, 1)
kazni_dict_entry("onesnaženje živil ali krme", 12, jail_time_for_kd, 1)
kazni_dict_entry("opravljanje", 1, jail_time_for_kd, 0.2)
kazni_dict_entry("opustitev nadzorstva v javnem prometu", 12, jail_time_for_kd, 1)
kazni_dict_entry("opustitev ovadbe kaznivega dejanja ali storilca", 3, jail_time_for_kd, 0)
kazni_dict_entry("opustitev ovadbe, da se pripravlja kaznivo dejanje", 3, jail_time_for_kd, 0)
kazni_dict_entry("opustitev pomoči", 1, jail_time_for_kd, 1)
kazni_dict_entry("opustitev zdravstvene pomoči", 1, jail_time_for_kd, 1)
kazni_dict_entry("organiziranje denarnih verig in nedovoljeno prirejanje iger na srečo", 8, jail_time_for_kd, 0.1)
kazni_dict_entry("overitev lažne vsebine", 3, jail_time_for_kd, 0)
kazni_dict_entry("oviranje pogreba in skrunitev groba", 3, jail_time_for_kd, 0)
kazni_dict_entry("oviranje pravosodnih in drugih državnih organov", 5, jail_time_for_kd, 0.1)
kazni_dict_entry("oviranje verskih obredov", 1, jail_time_for_kd, 0.1)
kazni_dict_entry("očitanje kaznivega dejanja z namenom zaničevanja", 0.5, jail_time_for_kd, 1)
kazni_dict_entry("oškodovanje javnih sredstev", 8, jail_time_for_kd, 0.5)
kazni_dict_entry("oškodovanje tujih pravic", 1, jail_time_for_kd, 1)
kazni_dict_entry("oškodovanje upnikov", 5, jail_time_for_kd, 0.5)
kazni_dict_entry("pomoč storilcu po storitvi kaznivega dejanja", 5, jail_time_for_kd, 0)
kazni_dict_entry("ponareditev ali uničenje poslovnih listin", 2, jail_time_for_kd, 0.1)
kazni_dict_entry("ponareditev ali uničenje uradne listine, knjige, spisa ali arhivskega gradiva", 3, jail_time_for_kd, 0.1)
kazni_dict_entry("ponareditev volilnih izidov", 2, jail_time_for_kd, 0)
kazni_dict_entry("ponarejanje denarja", 10, jail_time_for_kd, 0.4)
kazni_dict_entry("ponarejanje in uporaba ponarejenih vrednotnic ali vrednostnih papirjev", 10, jail_time_for_kd, 0.4)
kazni_dict_entry("ponarejanje listin", 3, jail_time_for_kd, 0.1)
kazni_dict_entry("ponarejanje znamenj za zaznamovanje blaga, mer in uteži", 2, jail_time_for_kd, 0.1)

# MARK LOBODA
kazni_dict_entry("posebni primeri ponarejanja listin", 1, jail_time_for_kd, 0)
kazni_dict_entry("posebno huda telesna poškodba", 15, jail_time_for_kd, 1)
kazni_dict_entry("posilstvo", 15, jail_time_for_kd, 1)
kazni_dict_entry("poslovna goljufija", 10, jail_time_for_kd, 0)
kazni_dict_entry("povzročitev nevarnosti", 2, jail_time_for_kd, 0.7)
kazni_dict_entry("povzročitev nevarnosti pri gradbeni dejavnosti", 8, jail_time_for_kd, 0.3)
kazni_dict_entry("povzročitev prometne nesreče iz malomarnosti", 8, jail_time_for_kd, 0.8)
kazni_dict_entry("povzročitev smrti iz malomarnosti", 5, jail_time_for_kd, 0.5)
kazni_dict_entry("povzročitev splošne nevarnosti", 15, jail_time_for_kd, 0.5)
kazni_dict_entry("povzročitev stečaja z goljufijo ali nevestnim poslovanjem", 8, jail_time_for_kd, 0)
kazni_dict_entry("poškodovanje ali uničenje javnih naprav", 5, jail_time_for_kd, 0.3)
kazni_dict_entry("poškodovanje ali uničenje stvari, ki so posebnega kulturnega pomena ali naravne vrednote", 8, jail_time_for_kd, 0)
kazni_dict_entry("poškodovanje tuje stvari", 5, jail_time_for_kd, 0.2)
kazni_dict_entry("požig", 10, jail_time_for_kd, 1)
kazni_dict_entry("pranje denarja", 10, jail_time_for_kd, 0)
kazni_dict_entry("prenašanje kužnih bolezni pri živalih in rastlinah", 2, jail_time_for_kd, 0.5)
kazni_dict_entry("prenašanje nalezljivih bolezni", 5, jail_time_for_kd, 0.5)
kazni_dict_entry("prepovedano prehajanje meje ali ozemlja države", 8, jail_time_for_kd, 0)
kazni_dict_entry("preprečitev ali oviranje javnega shoda", 2, jail_time_for_kd, 0)
kazni_dict_entry("preprečitev dokazovanja", 5, jail_time_for_kd, 0)
kazni_dict_entry("preprečitev uradnega dejanja ali maščevanje uradni osebi", 5, jail_time_for_kd, 0.5)
kazni_dict_entry("preprečitev vrnitve na delo", 1, jail_time_for_kd, 0)
kazni_dict_entry("preslepitev kupcev", 2, jail_time_for_kd, 0.2)
kazni_dict_entry("preslepitev pri poslovanju z vrednostnimi papirji", 5, jail_time_for_kd, 0)
kazni_dict_entry("preslepitev pri pridobitvi in uporabi posojila ali ugodnosti", 3, jail_time_for_kd, 0.1)
kazni_dict_entry("pridobivanje oseb, mlajših od petnajst let, za spole namene", 1, jail_time_for_kd, 0.5)
kazni_dict_entry("prikazovanje, izdelava, posest in posredovanje pornografskega gradiva", 8, jail_time_for_kd, 0)
kazni_dict_entry("prikrivanje", 5, jail_time_for_kd, 0)
kazni_dict_entry("prisiljenje", 2, jail_time_for_kd, 0.7)
kazni_dict_entry("prisilna sklenitev zakonske zveze ali vzpostavitev podobne skupnosti", 5, jail_time_for_kd, 0)
kazni_dict_entry("proizvodnja in promet škodljivih sredstev za zdravljenje", 10, jail_time_for_kd, 0.8)
kazni_dict_entry("proizvodnja in promet škodljivih živil in drugih izdelkov", 12, jail_time_for_kd, 0.7)
kazni_dict_entry("protipraven odvzem prostosti", 5, jail_time_for_kd, 0.8)
kazni_dict_entry("protipravna prilastitev stvari ob preiskavi ali izvršbi", 8, jail_time_for_kd, 0)
kazni_dict_entry("protipravno omejevanje konkurence", 5, jail_time_for_kd, 0)
kazni_dict_entry("protipravno zavzetje nepremičnine", 1, jail_time_for_kd, 0.5)
kazni_dict_entry("protizakonito, pristransko in krivično sojenje", 3, jail_time_for_kd, 0)

# NAPREJ FJ
kazni_dict_entry("razžalitev", 0.5, jail_time_for_kd, 1)
kazni_dict_entry("rop", 15, jail_time_for_kd, 1)
kazni_dict_entry("roparska tatvina", 15, jail_time_for_kd, 1)
kazni_dict_entry("samovoljnost", 3, jail_time_for_kd, 0.5)
kazni_dict_entry("skrunitev trupla", 2, jail_time_for_kd, 0)
kazni_dict_entry("sodelovanje pri pretepu", 1, jail_time_for_kd, 1)
kazni_dict_entry("sodelovanje v skupini, ki prepreči uradni osebi uradno dejanje", 3, jail_time_for_kd, 0.3)
kazni_dict_entry("sodelovanje v skupini, ki stori kaznivo dejanje", 5, jail_time_for_kd, 1)
kazni_dict_entry("spolna zloraba slabotne osebe", 8, jail_time_for_kd, 0.5)
kazni_dict_entry("spolni napad na osebo mlajšo od 14 let", 15, jail_time_for_kd, 1)
kazni_dict_entry("spolni napad na osebo, mlajšo od petnajst let", 15, jail_time_for_kd, 1)
kazni_dict_entry("spolno nasilje", 15, jail_time_for_kd, 1)
kazni_dict_entry("sprejemanje koristi za nezakonito posredovanje", 6, jail_time_for_kd, 0.5)
kazni_dict_entry("sprememba rodbinskega stanja", 3, jail_time_for_kd, 0.5)
kazni_dict_entry("sramotitev", 1, jail_time_for_kd, 0) # No data at all?
kazni_dict_entry("sramotitev slovenskega naroda ali narodnih skupnosti", 1, jail_time_for_kd, 0.2)
kazni_dict_entry("tatvina", 3, jail_time_for_kd, 1)
kazni_dict_entry("terorizem", 15, jail_time_for_kd, 1)
kazni_dict_entry("tihotapstvo", 15, jail_time_for_kd, 0.2)
kazni_dict_entry("trgovina z ljudmi", 15, jail_time_for_kd, 0.9)
kazni_dict_entry("uboj", 15, jail_time_for_kd, 0.9)
kazni_dict_entry("ugrabitev", 10, jail_time_for_kd, 1)
kazni_dict_entry("umor", 15, jail_time_for_kd, 1)
kazni_dict_entry("uničenje ali ponareditev volilnih listin", 2, jail_time_for_kd, 0)
kazni_dict_entry("uničevanje gozdov", 3, jail_time_for_kd, 0)
kazni_dict_entry("upor oseb, ki jim je vzeta prostost", 3, jail_time_for_kd, 0)
kazni_dict_entry("uporaba ponarejenega negotovinskega plačilnega sredstva", 8, jail_time_for_kd, 0.6)

# ML naprej do konca
kazni_dict_entry("velika tatvina", 8, jail_time_for_kd, 1)
kazni_dict_entry("vojna hudodelstva", 15, jail_time_for_kd, 0.1)
kazni_dict_entry("zalezovanje", 3, jail_time_for_kd, 0.9)
kazni_dict_entry("zanemarjanje mladoletne osebe in surovo ravnanje", 5, jail_time_for_kd, 1)
kazni_dict_entry("zaposlovanje na črno", 3, jail_time_for_kd, 0.9)
kazni_dict_entry("zapustitev poškodovanca v prometni nesreči brez pomoči", 5, jail_time_for_kd, 0.8)
kazni_dict_entry("zapustitev slabotne osebe", 2, jail_time_for_kd, 0.1)
kazni_dict_entry("zatajitev", 5, jail_time_for_kd, 0.9)
kazni_dict_entry("zloraba informacijskega sistema", 5, jail_time_for_kd, 0.1)
kazni_dict_entry("zloraba izvršbe", 2, jail_time_for_kd, 0.1)
kazni_dict_entry("zloraba negotovinskega plačilnega sredstva", 8, jail_time_for_kd, 0.5)
kazni_dict_entry("zloraba notranje informacije", 5, jail_time_for_kd, 0.1)
kazni_dict_entry("zloraba osebnih podatkov", 5, jail_time_for_kd, 0.7)
kazni_dict_entry("zloraba položaja ali zaupanja pri gospodarski dejavnosti", 8, jail_time_for_kd, 0.7)
kazni_dict_entry("zloraba pooblastil", 8, jail_time_for_kd, 0.7)
kazni_dict_entry("zloraba pravic iz socialnega zavarovanja", 2, jail_time_for_kd, 0.5)
kazni_dict_entry("zloraba prostitucije", 12, jail_time_for_kd, 0.1)
kazni_dict_entry("zloraba uradnega položaja ali uradnih pravic", 8, jail_time_for_kd, 0.7)
kazni_dict_entry("zloraba volilne pravice", 1, jail_time_for_kd, 0.1)
kazni_dict_entry("zloraba znamenj za pomoč in nevarnost", 5, jail_time_for_kd, 0.2)
kazni_dict_entry("šikaniranje na delovnem mestu", 3, jail_time_for_kd, 1)
kazni_dict_entry("ščuvanje in javno poveličevanje terorističnih dejanj", 10, jail_time_for_kd, 0.7)
kazni_dict_entry("ščuvanje k nasilni spremembi ustavne ureditve", 5, jail_time_for_kd, 0.2)
kazni_dict_entry("žaljiva obdolžitev", 1, jail_time_for_kd, 0.1)

In [15]:
# write kazni_dict to json file and NaN as null
with open('../data/kazni_dict.json', 'w') as f:
    json.dump(kazni_dict, f, indent=4, sort_keys=True, default=str)

    
        